# Building on top of low-level NVFlare APIs

In this chapter, we will explore some of the low-level APIs available in NVFlare to build fully custom components and algorithms.

### P2P distributed optimization as a guiding example

As a guiding example, we will walk through how to implement from scratch the `nvflare.app_opt.p2p` module, which implements peer-to-peer (P2P) distributed optimization algorithms.

In peer-to-peer (P2P) algorithms, being distributed optimization or decentralized federated learning algorithms, clients communicate directly with each other without relying on a central server to aggregate updates. Implementing P2P algorithms usually requires careful orchestration to handle communication, synchronization, and data exchange among clients.

Thankfully, NVFlare natively provides the primitives to easily build such a system. In fact, it natively supports various communication and orchestration patterns, including peer-to-peer interactions, which are crucial for decentralized algorithms.

### What to expect

- In [section 1](../09.1_controllers/building_a_custom_controller.ipynb) we will look at [Controllers](https://nvflare.readthedocs.io/en/2.5/apidocs/nvflare.apis.impl.controller.html#module-nvflare.apis.impl.controller), components that manage job execution and orchestrate [tasks](https://nvflare.readthedocs.io/en/2.5/apidocs/nvflare.apis.controller_spec.html#nvflare.apis.controller_spec.Task). We will show how to build a custom server-side controller, able to configure multiple clients and orchestrate their execution of P2P algorithms, via some of its methods, like `broadcast_and_wait` or `send_and_wait`.
- In [section 2](../09.2_executors/building_custom_executors.ipynb) we will look at [Executors](https://nvflare.readthedocs.io/en/2.5/apidocs/nvflare.apis.executor.html#module-nvflare.apis.executor), components that perform computations and handle tasks received from the controller. We will show how to build a custom client-side executor, able to perform P2P communication and orchestration. To implement P2P interactions, we will exploit sending [messages via aux channes](https://nvflare.readthedocs.io/en/2.5/apidocs/nvflare.private.aux_runner.html#nvflare.private.aux_runner.AuxRunner.send_aux_request), a Flare feature that enables direct communication between clients.


### Prerequisite: Pythonic configs

Before we dive into the controller and executor, let's start by creating a Pythonic way to define the configuration of our network. We'll start by simply defining a `Node` (i.e. a client in the network), its `Neighbors` (i.e. other clients with which a client communicates, each with a weight) and combine them to define a `Network` (i.e. a network of clients with neighbors).

```python
from dataclasses import dataclass, field

@dataclass
class Neighbor:
    id: int | str
    weight: float | None = None

@dataclass
class Node:
    id: int | str
    neighbors: list[Neighbor] = field(default_factory=list)

@dataclass
class Network:
    nodes: list[Node] = field(default_factory=list)
```

Then we'll define a global and a local config objects to be passed to the controller and executors respectively.

```python
@dataclass
class Config:
    network: Network
    extra: dict = field(default_factory=dict)

@dataclass
class LocalConfig:
    neighbors: list[Neighbor]
    extra: dict = field(default_factory=dict)
```

The `extra` parameter can be used to pass additional parameters, usually specific for the various algorithms. 

To actual implementation of the objects above can be found in `nvflare/app_opt/p2p/types/__init__.py` (you'll see they'll have the `__dict__` and `__post_init__` methods defined facilitate serializing and deserializing them, which is needed for NVFlare).

Here's an example of a ring network with 3 clients, running an algorithm for 100 iterations:
```shell
Config(
    extra={"iterations":100},
    network=Network(
        nodes=[
            Node(
                id='site-1',
                neighbors=[
                    Neighbor(id='site-2', weight=0.1),
                ]
            ),
            Node(
                id='site-2',
                neighbors=[
                    Neighbor(id='site-3', weight=0.1),
                ]
            ),
            Node(
                id='site-3',
                neighbors=[
                    Neighbor(id='site-1', weight=0.1),
                ]
            ),
        ]
    )
)
```